# Emailing Participants with API

This notebook is used to email participants with their API keys. This is not to be used by participants themselves, but rather by the workshop organizers to send out keys.

However, you can take a look at the code to see how bulk e-mailing is done using `sendgrid` 

Sendgrid (from Twilio) is a service that allows you to send emails in bulk. It is a paid service, but it has a free tier that allows you to send up to 100 emails per day.
You can use it to send emails to participants with their API keys.

You can sign up for a free account at [SendGrid](https://sendgrid.com/).

There are other services that allow you to send emails in bulk, such as [Mailgun](https://www.mailgun.com/) and [Amazon SES](https://aws.amazon.com/ses/), but we will use SendGrid for this workshop.

In old days this was done using `smtplib` and `email` libraries, however, these days most e-mail providers have limits on how many emails you can send per day, so it is better to use a service that is designed for this purpose.
This notebook will show you how to use SendGrid to send emails to participants with their API keys

In [ ]:
import os
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail

# Store your key securely (e.g., using environment variable)
SENDGRID_API_KEY = os.getenv("SENDGRID_API")  # or paste directly (not recommended)
if not SENDGRID_API_KEY:
    raise ValueError("SENDGRID_API_KEY environment variable not set")
else:
    print("SendGrid API key is set.") # again we do not want to print the key itself!! then anyone could use it....
sender_email = "valdis.saulespurens@lnb.lv"
print("Sender email is set to:", sender_email)


SendGrid API key is set.
Sender email is set to: valdis.saulespurens@lnb.lv


## Reading e-mail participants from JSON file

For convenience, we will read the participants from a JSON file that is stored in the `temp` directory of our parent folder. This file should contain a list of participants with their names, emails, and API keys.
We also have a role key as well for each participant such as `student`, `teacher`, etc. 
Sample file of two participants looks like this:

```json
[
    {
        "name": "John Doe",
        "email": "john.doe@example.com",
        "key": "sk-1234567890abcdef",
        "role": "student"
    },
    {
        "name": "Jane Smith",
        "email": "jane.smith@example.org",
        "key": "sk-abcdef1234567890",
        "role": "teacher"
    }
]
```  

In [ ]:
# next lets load participants from emails.json in temp directory of our parent folder
import json
from pathlib import Path
email_file = Path("../temp/emails.json") # note I am keeping this file in temp directory as we do not want user data to be committed to this repository
if not email_file.exists():
    raise FileNotFoundError(f"Email file not found: {email_file}")
with open(email_file, 'r') as f:
    participants = json.load(f)
print(f"Loaded {len(participants)} participants from {email_file}")

Loaded 4 participants from ..\temp\emails.json


In [5]:
# print keys (not values) of first participant
if participants:
    print("Keys of first participant:", list(participants[0].keys()))

Keys of first participant: ['name', 'email', 'role', 'key']


In [ ]:
# we have 4 keys for each participant: "name", "email", "role" and "key"
# now we will send an email to each participant with their unique API key

for participant in participants:
    message = Mail(
        from_email=sender_email,
        to_emails=participant["email"],
        subject="Testing Your Unique API Key for the BSSDH Workshop",
        plain_text_content=(
            f"Dear {participant['name']},\n\n"
            f"Thank you for participating in our LLM using API workshop. Your unique API key is: {participant['key']} - for now this is a test\n\n"
            "Please keep this key secure and do not share it with others.\n\n"
            "Best regards,\n"
            "On behalf of the BSDDH Workshop Team - Valdis Saulespurens\n"

        )
    )
    try:
        sg = SendGridAPIClient(SENDGRID_API_KEY)
        response = sg.send(message)
        print(f"✅ Sent to {participant['email']} – Status: {response.status_code}")
    except Exception as e:
        print(f"❌ Failed to send to {participant['email']}: {e}")

# SendGrid does not require explicit cleanup - however running Sendgrid from Notebook causes spinning wheel in VSCode
# TODO read more SendGrid documentation about cleanup
# URL: https://www.twilio.com/docs/sendgrid/for-developers/sending-email/quickstart-python
# so we will delete the SendGrid client object to close the connection - this is not strictly necessary
del sg
import gc
gc.collect()
print("All emails processed.")
